In [50]:
import pandas as pd
import time
import random
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


In [8]:
# List of all league-season URLs
leagues = {
    '9': 'Premier League',
    '12': 'La Liga',
    '11': 'Serie A',
    '20': 'Bundesliga',
    '13': 'Ligue 1',
    '10': 'Championship',
    '19': 'Major League Soccer',
    '24': 'Brasileirao',
    '31': 'Liga MX',
    '23': 'Eredivisie',
    '32': 'Primeira Liga',
    '39': 'Liga Profesional',
    '37': 'Belgian Pro League',
    '40': 'Serie B'
}

In [75]:
# Seasons
seasons = [f'{year}-{year+1}' for year in range(2013, 2024)]

In [76]:
seasons

['2013-2014',
 '2014-2015',
 '2015-2016',
 '2016-2017',
 '2017-2018',
 '2018-2019',
 '2019-2020',
 '2020-2021',
 '2021-2022',
 '2022-2023',
 '2023-2024']

In [24]:
# URL template: per-player stats
base_url = 'https://fbref.com/en/comps/{code}/{season}/stats/{season}-{league_name}-Stats'
url_list = []

for code, name in leagues.items():
    for season in seasons:
        league_name_url = name.replace(" ", "-").replace(".", "")
        url = base_url.format(code=code, season=season, league_name=league_name_url)
        url_list.append((name, season, url))

In [77]:
base_url = 'https://fbref.com/en/comps/{code}/{season}/stats/{season}-{league_name}-Stats'
last_season = []

for code, name in leagues.items():
        season = '2024-2025'
        league_name_url = name.replace(" ", "-").replace(".", "")
        url = base_url.format(code=code, season=season, league_name=league_name_url)
        last_season.append((name, season, url))

In [82]:
# --- SELENIUM SETUP ---
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


In [62]:
# --- SCRAPING LOOP ---
all_data = []

for league, season, url in tqdm(url_list):
    try:
        print(f"\n🔍 Scraping {league} {season}...")

        driver.get(url)
        time.sleep(random.uniform(5, 7))  # Wait for JS to load

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table = soup.find('table', {'id': 'stats_standard'})
        if table is None:
            print(f"⚠️ No player stats table found at {url}")
            continue

        df = pd.read_html(str(table), header=[0, 1])[0]
        df.columns = [' '.join(col).strip() if isinstance(col, tuple) else col for col in df.columns]
        df.columns = [ col.replace("Unnamed: ", "").split(" ", 1)[-1] if col.startswith("Unnamed: ") else col for col in df.columns ]
        df = df[df['Player'].values != 'Player']  # Remove internal header rows
        df['League'] = league
        df['Season'] = season
        all_data.append(df)

        print(f"✅ {len(df)} players scraped.")
        time.sleep(random.uniform(1, 3))

    except Exception as e:
        print(f"❌ Error scraping {league} {season}: {e}")
        time.sleep(random.uniform(3, 6))

# --- CLEANUP ---
driver.quit()

# --- RESULT ---
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"\n🎉 Done! Collected {combined_df.shape[0]} player-season rows across {len(all_data)} tables.")
else:
    print("🚫 No data was scraped.")

  0%|          | 0/154 [00:00<?, ?it/s]


🔍 Scraping Premier League 2013-2014...


C:\Users\User\AppData\Local\Temp\ipykernel_174700\2003302705.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), header=[0, 1])[0]


✅ 561 players scraped.


  1%|          | 1/154 [00:09<23:02,  9.03s/it]

...


🔍 Scraping Serie B 2023-2024...


C:\Users\User\AppData\Local\Temp\ipykernel_174700\2003302705.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), header=[0, 1])[0]


✅ 360 players scraped.


100%|██████████| 154/154 [32:36<00:00, 12.71s/it]



🎉 Done! Collected 76502 player-season rows across 143 tables.


In [67]:
combined_df[['Player','League','Season']]

,Player,League,Season
0,Charlie Adam,Premier League,2013-2014
1,Emmanuel Adebayor,Premier League,2013-2014
2,Adrián,Premier League,2013-2014
3,Gabriel Agbonlahor,Premier League,2013-2014
4,Daniel Agger,Premier League,2013-2014
...,...,...,...
76497,Scott Wright,Serie B,2023-2024
76498,Tete Yengi,Serie B,2023-2024
76499,Rıdvan Yılmaz,Serie B,2023-2024
76500,Elie Youan,Serie B,2023-2024


In [69]:
combined_df.loc[combined_df['Player'] == 'Manor Solomon'][['Player','League','Season']]

,Player,League,Season
5336,Manor Solomon,Premier League,2022-2023
5917,Manor Solomon,Premier League,2023-2024
64727,Manor Solomon,Liga Profesional,2018-2019
65120,Manor Solomon,Liga Profesional,2019-2020
65579,Manor Solomon,Liga Profesional,2020-2021
66009,Manor Solomon,Liga Profesional,2021-2022


In [83]:
last_season_list = []
for league, season, url in tqdm(last_season):
    try:
        print(f"\n🔍 Scraping {league} '2024-2025'...")

        driver.get(url)
        time.sleep(random.uniform(5, 7))  # Wait for JS to load

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table = soup.find('table', {'id': 'stats_standard'})
        if table is None:
            print(f"⚠️ No player stats table found at {url}")
            continue

        df = pd.read_html(str(table), header=[0, 1])[0]
        df.columns = [' '.join(col).strip() if isinstance(col, tuple) else col for col in df.columns]
        df.columns = [ col.replace("Unnamed: ", "").split(" ", 1)[-1] if col.startswith("Unnamed: ") else col for col in df.columns ]
        df = df[df['Player'].values != 'Player']  # Remove internal header rows
        df['League'] = league
        df['Season'] = season
        last_season_list.append(df)

        print(f"✅ {len(df)} players scraped.")
        time.sleep(random.uniform(1, 3))

    except Exception as e:
        print(f"❌ Error scraping {league} {season}: {e}")
        time.sleep(random.uniform(3, 6))

# --- CLEANUP ---
driver.quit()

# --- RESULT ---
if last_season_list:
    last_season_df = pd.concat(last_season_list, ignore_index=True)
    print(f"\n🎉 Done! Collected {combined_df.shape[0]} player-season rows across {len(all_data)} tables.")
else:
    print("🚫 No data was scraped.")

  0%|          | 0/14 [00:00<?, ?it/s]


🔍 Scraping Premier League '2024-2025'...


C:\Users\User\AppData\Local\Temp\ipykernel_174700\2245004445.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), header=[0, 1])[0]


✅ 574 players scraped.


  7%|▋         | 1/14 [00:14<03:06, 14.36s/it]

...


🔍 Scraping Serie B '2024-2025'...


C:\Users\User\AppData\Local\Temp\ipykernel_174700\2245004445.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), header=[0, 1])[0]


✅ 372 players scraped.


100%|██████████| 14/14 [03:17<00:00, 14.09s/it]



🎉 Done! Collected 76502 player-season rows across 143 tables.


In [89]:
last_season_df.loc[last_season_df['League'] == 'Belgian Pro League']['Squad'].unique()

array(['Gent', 'Cercle Brugge', 'Antwerp', 'Dender', 'Genk', 'Anderlecht',
       'OH Leuven', 'Westerlo', 'Kortrijk', 'Standard Liège',
       'Sint-Truiden', 'Charleroi', 'Mechelen', 'Union SG', 'Beerschot',
       'Club Brugge'], dtype=object)

In [3]:
combined_df.loc[combined_df['Player'] == 'Anan Khalaili']

NameError: name 'combined_df' is not defined

In [2]:
last_season_df.loc[last_season_df['Squad'] == 'Union SG'].sort_values(by='Per 90 Minutes xG+xAG',ascending=False)

NameError: name 'last_season_df' is not defined

In [1]:
last_season_df.to_csv('last_season.csv')

NameError: name 'last_season_df' is not defined